*By DJun (github.com/djun); 2021-8-12*

### 背景

#### 为什么学爬虫
- 简单容易上手
- 练习写Python代码的极佳方式
- 学习了解抓取用来分析的数据的方法
- 抓取网络上我们感兴趣的信息，加以利用

#### “爬虫”是什么
- 本质是从网络上爬取公开数据
- 程序根据我们预先设定好的规则，沿着线索爬取所需数据
- 例子：抓取 新闻列表 -> 每篇新闻正文

### 准备工作

#### 工具
- Anaconda 3（推荐，大部分所需模块，Anaconda已帮我们准备好）
- Jupyter Notebook（推荐在学习研究时使用，适合“看一下走一步”）
- PyCharm（推荐在开发项目时使用）

#### 整体流程
- 从网络上获取原始数据（网页，RestfulAPI，WebSocket，等）
- 经过特定的解析（解析HTML/CSS/JavaScript，JSON，XML，长文本，其他特殊格式）
- 最终整理为我们所需的数据（真正需要的能为我所用的内容）

#### 常用分析工具
- 【重点讲】浏览器的开发者工具（在浏览器中按F12调出）
- Fiddler（个人使用免费）
- Charles（付费）

*注1：F12工具一般在浏览器中自带，常用“元素”（查看页面HTML代码、CSS样式、事件），“控制台”（通过JavaScript进行测试），“源”（查看运行在页面上的JavaScript等源代码），“网络”（类似Fiddler、Charles中的页面加载监控功能）。*

*注2：Fiddler、Charles的原理，都是在本机开一个代理服务器，浏览器或者其他软件通过这个本机代理访问网络，它们就可以把过程中的HTTP数据抓取下来；推荐在了解清楚大致原理后再使用，推荐使用Fiddler。*

#### 相关Python模块（最小化）
- 访问网络：【重点讲】requests（简单http操作）；其他有 selenium（通过浏览器模拟网页操作）
- 解析数据：【重点讲】lxml（解析HTML、XML）；其他有 BeautifulSoup（解析HTML），自带模块json（解析JSON），自带模块re（正则表达式，提取文本），js2py（解析JavaScript）
- 持久储存数据：自带的文件操作open（直接写入文本文件），自带的csv（写入csv文本文件），【下次重点讲】自带的sqlite3（写入轻量级文件型数据库）

### 参考资料

#### 教程与文档
- XPath教程（使用lxml通过编写XPath规则来提取所需数据） https://www.runoob.com/xpath/xpath-syntax.html
- requests文档（推荐使用的HTTP客户端模块，热门，资料多；当然也可以用自带的urllib，但并不好用） https://docs.python-requests.org/zh_CN/latest/

#### 陈同学的博客，可参考思路和写法
- 爬取新闻页面中的网页链接 https://blog.csdn.net/c348762444/article/details/118694793
- 爬取网页数据并导入excel表格 https://blog.csdn.net/c348762444/article/details/118701879
- 尝试先爬取新闻链接，然后爬取链接后的正文（XPath） https://blog.csdn.net/c348762444/article/details/119170469

#### 其他博客
- 使用Python爬取多篇各类新闻文章（beautifulsoup） https://blog.csdn.net/qq_43179991/article/details/103243743

### 实操演示

#### 例子1：爬取网易新闻标题，下载新闻封面图片

In [1]:
# 先用浏览器打开看看页面长什么样
# https://3g.163.com/touch/news

import requests

# HTTP操作，最最常用的有GET、POST这两种方法；大致原理见百度百科
# ref: https://baike.baidu.com/item/HTTP/243074

# 我们用浏览器打开一个网页，本质上就是“客户端”通过“GET方法”获取到下面结果输出的这些内容
# 这里使用requests简单快速操作
# requests的具体用法，详见它的官方文档
r = requests.get("https://3g.163.com/touch/news")

# 设置文本编码
r.encoding = "utf-8"

# 查看解码后的文本内容（就是HTML代码）
r.text

'<!DOCTYPE html>\n<html lang="en">\n  <head>\n    <meta charset="UTF-8" />\n    <link rel="dns-prefetch" href="//cms-bucket.nosdn.127.net" />\n    <link rel="dns-prefetch" href="//pic-bucket.nosdn.127.net" />\n    <link rel="dns-prefetch" href="//static.ws.126.net" />\n    <meta name="viewport" content="width=device-width, initial-scale=1.0" />\n    <meta http-equiv="X-UA-Compatible" content="ie=edge" />\n    <title>网易新闻_手机网易网</title>\n    <meta content="telephone=no" name="format-detection" />\n    <meta name="keywords" content="新闻,网易,163,网易新闻,新闻中心,新闻频道,时事,报道,时政,国际,国内,社会,聚焦,评论,文化,教育,深度,网评,专题,环球,传播,论坛,图片,军事,焦点,排行,环保,校园,法治" />\n    <meta name="description" content="提供时政新闻,国内新闻,国际新闻,社会新闻,时事评论,新闻图片,新闻专题,新闻论坛,军事,历史等最热门一手的新闻资讯。手机网易网新闻频道-https://3g.163.com/touch/news" />\n    <meta name="google-site-verification" content="vDJDt0eLizo98mTqFjhG4ONEm8DlFI7bdonVyRDi-EY" />\n    <script type=\'text/javascript\'>window._ANT_PROJECT_ID="NTM-5AE0KFYY-2";(function(){var domainMatches=[{test:/163\\.co

In [2]:
# 用lxml或BeautifulSoup来解析这一堆HTML代码
# HTML是xml的子集，是一种树形结构的数据（想一想《数据结构》里面的“树”），详见百度百科
# ref: https://baike.baidu.com/item/HTML/97049

# 网络资料有两个误区：
# 使用lxml.etree  ——解析HTML时使用lxml.html更合适
# 使用re 正则表达式  ——比较难用，非必要时没必要用更纠结的工具 走弯路来处理像HTML这种有规范的结构化数据

# 这里用lxml演示

from lxml import html

# 将HTML代码转换为HtmlElement（树节点对象）
# 目前拿到的tree对象，其实就是根节点的对象
# 查找我们所需的信息，一般也是从根节点开始查找
tree = html.fromstring(r.text)
type(tree)

lxml.html.HtmlElement

In [3]:
# 【重点讲】编写XPath提取数据

# 先在F12窗口中查看元素，找到需要的信息的规律后，来编写XPath规则
# tree.xpath()可以根据我们写的规则，返回我们需要的信息

# 提取元素对象列表
el1 = tree.xpath("//div[contains(@class, 'tab-content')]//article/a")
print(el1)
print()

el2 = tree.xpath("//div[contains(@class, 'tab-content')]//article//h3/text()")
# 提取元素中的文本内容
for i in el2:
    print(i)
print()

el3 = tree.xpath("//div[contains(@class, 'tab-content')]//article/a/@href")
# 提取元素中某个具体属性的值
for i in el3:
    print(i)

[<Element a at 0x68cbbd8>, <Element a at 0x68cbc28>, <Element a at 0x68cbc78>, <Element a at 0x68cbcc8>, <Element a at 0x68cbd18>, <Element a at 0x68cbd68>, <Element a at 0x68cbdb8>, <Element a at 0x68cbe08>, <Element a at 0x68cbe58>, <Element a at 0x68cbea8>, <Element a at 0x68cbef8>, <Element a at 0x68cbf48>, <Element a at 0x68cbf98>, <Element a at 0x68da048>, <Element a at 0x68da098>, <Element a at 0x68da0e8>, <Element a at 0x68da138>, <Element a at 0x68da188>, <Element a at 0x68da1d8>, <Element a at 0x68da228>, <Element a at 0x68da278>, <Element a at 0x68da2c8>, <Element a at 0x68da318>, <Element a at 0x68da368>, <Element a at 0x68da3b8>, <Element a at 0x68da408>, <Element a at 0x68da458>, <Element a at 0x68da4a8>, <Element a at 0x68da4f8>, <Element a at 0x68da548>, <Element a at 0x68da598>, <Element a at 0x68da5e8>, <Element a at 0x68da638>, <Element a at 0x68da688>, <Element a at 0x68da6d8>, <Element a at 0x68da728>, <Element a at 0x68da778>, <Element a at 0x68da7c8>, <Element a 

In [4]:
# 提取图片，下载到本机

# 先把页面上 每个新闻的封面图片链接 提取出来
img_list = tree.xpath("//article//img/@src")
img_list

['//cms-bucket.ws.126.net/2021/0813/e341a259p00qxr93z005ic000s600e3c.png?imageView&thumbnail=234y146&quality=45&interlace=1&enlarge=1&type=png',
 '//cms-bucket.ws.126.net/2021/0813/f0a68abdj00qxr7rh001hc000s600e3c.jpg?imageView&thumbnail=234y146&quality=45&interlace=1&enlarge=1&type=png',
 '//cms-bucket.ws.126.net/2021/0813/60622c6bp00qxr5qp004vc0009c0070c.png?imageView&thumbnail=234y146&quality=45&interlace=1&enlarge=1&type=png',
 '//cms-bucket.ws.126.net/2021/0813/8715d063p00qxr72g0083c0009c0070c.png?imageView&thumbnail=234y146&quality=45&interlace=1&enlarge=1&type=png',
 '//bjnewsrec-cv.ws.126.net/little34384c6df66j00qxprwa004fd000u001cap.jpg?imageView&thumbnail=234y146&quality=45&interlace=1&enlarge=1&type=png',
 '//cms-bucket.ws.126.net/2021/0812/8d24be8ep00qxqd4s001cc0009c0070c.png?imageView&thumbnail=234y146&quality=45&interlace=1&enlarge=1&type=png',
 '//cms-bucket.ws.126.net/2021/0812/9345b43ap00qxq9t2001ac0009c0070c.png?imageView&thumbnail=234y146&quality=45&interlace=1&enlar

In [5]:
import os

# 准备输出目录
OUTPUT_DIR = "images"
os.makedirs(OUTPUT_DIR, exist_ok=True)

In [6]:
from urllib.parse import urljoin, urlparse

# 预处理图片链接（抓取到的是图片的相对路径，要用原网址 即我们一开始访问的那个网址，通过urllib.parse.urljoin修正为完整链接）
new_img_list = []
for i in img_list:
    img = urljoin("https://3g.163.com/touch/news", i)
    new_img_list.append(img)
new_img_list

['https://cms-bucket.ws.126.net/2021/0813/e341a259p00qxr93z005ic000s600e3c.png?imageView&thumbnail=234y146&quality=45&interlace=1&enlarge=1&type=png',
 'https://cms-bucket.ws.126.net/2021/0813/f0a68abdj00qxr7rh001hc000s600e3c.jpg?imageView&thumbnail=234y146&quality=45&interlace=1&enlarge=1&type=png',
 'https://cms-bucket.ws.126.net/2021/0813/60622c6bp00qxr5qp004vc0009c0070c.png?imageView&thumbnail=234y146&quality=45&interlace=1&enlarge=1&type=png',
 'https://cms-bucket.ws.126.net/2021/0813/8715d063p00qxr72g0083c0009c0070c.png?imageView&thumbnail=234y146&quality=45&interlace=1&enlarge=1&type=png',
 'https://bjnewsrec-cv.ws.126.net/little34384c6df66j00qxprwa004fd000u001cap.jpg?imageView&thumbnail=234y146&quality=45&interlace=1&enlarge=1&type=png',
 'https://cms-bucket.ws.126.net/2021/0812/8d24be8ep00qxqd4s001cc0009c0070c.png?imageView&thumbnail=234y146&quality=45&interlace=1&enlarge=1&type=png',
 'https://cms-bucket.ws.126.net/2021/0812/9345b43ap00qxq9t2001ac0009c0070c.png?imageView&thum

In [7]:
# 下载图片（获取图片的原始数据，通过open()写入文件）
for i in new_img_list:
    r = requests.get(i)  # 还是通过GET方法，获取图片数据
    filename = os.path.basename(urlparse(i).path)  # 解析出链接中的文件名
    
    # 拼接路径，写到指定输出目录下
    with open(os.path.join(OUTPUT_DIR, filename), "wb") as fp:
        fp.write(r.content)  # 与r.text的解码文本不同，r.content是原始的未解码数据

os.listdir(OUTPUT_DIR)  # 列出目录下的文件名

['02deed23p00qxq8j10036c0009c0070c.png',
 '0b495c28p00qxpyj10010c0009c0070c.png',
 '0e37b5455c5040db8537f2a6555dede4.jpeg',
 '14c675fbj00qxpyeh0020d000ip00e1p.jpg',
 '185c4deap00qxra2f0040c0009c0070c.png',
 '1885d256j00qxok6c001zc000go00cim.jpg',
 '2dfb6c34p00qxrbe2004vc0009c0070c.png',
 '39f53c70p00qxq3ae0036c0009c0070c.png',
 '3b349ad6b7174f78afc3fff6028c66c5.jpeg',
 '3dc6e8edp00qxpy4y007cc0009c0070c.png',
 '532515ba453b47469b2b174af758418c.jpeg',
 '59752005p00qxr2n7000wc000s600e3c.png',
 '5a3bc791p00qxpqw90014c0009c0070c.png',
 '60622c6bp00qxr5qp004vc0009c0070c.png',
 '61405aa4p00qxr579002mc000s600e3c.png',
 '8715d063p00qxr72g0083c0009c0070c.png',
 '8d24be8ep00qxqd4s001cc0009c0070c.png',
 '8ffcccd1p00qxq1gg001dc0009c0070c.png',
 '9345b43ap00qxq9t2001ac0009c0070c.png',
 '9de204ebp00qxpubr000yc0009c0070c.png',
 'aa33e881p00qxpwwq001kc0009c0070c.png',
 'abd76a20p00qxr2lq004nc0009c0070c.png',
 'ada886b8e75f4b36be972b10b1b89de7.jpeg',
 'b1190b8af33c41c697f1c62466e7db1c.jpeg',
 'bd493219p

#### 例子2：把抓取到的内容写入文本文件、csv文件（csv类似表格）

*推荐使用EmEditor、UltraEdit等编辑器来查看csv文件*

In [9]:
# 继续用上面现成的新闻数据

# 获取新闻标题
news_titles = tree.xpath("//div[contains(@class, 'tab-content')]//article//h3/text()")
# 获取新闻链接
news_links = tree.xpath("//div[contains(@class, 'tab-content')]//article/a/@href")
news_links = [urljoin("https://3g.163.com/touch/news", i) for i in news_links]  # 还原为完整链接

print(len(news_titles), len(news_links))  # 数量一致，大概率说明标题和链接是对应的（这里没有打算做验证，是不够严谨的哈）

44 44


In [10]:
# 演示：写入到文本文件

# 写入
with open("news.txt", "w", encoding="utf-8") as fp:
    for title, link in zip(news_titles, news_links):
        fp.write(f"{title} (链接：{link})\n")

# 重新读取出来查看
with open("news.txt", "r", encoding="utf-8") as fp:
    lines = fp.readlines()
    for i in lines:
        print(i)

治国理政新实践 | 绿色绘出幸福底色 (链接：https://3g.163.com/news/article/GH95GLOJ000189FH.html?clickfrom=channel2018_news_newsList#offset=0)

美国是当之无愧的全球第一抗疫失败国 (链接：https://3g.163.com/news/article/GH95I3F0000189FH.html?clickfrom=channel2018_news_newsList#offset=1)

乌合麒麟发布新作，每个细节都是梗！ (链接：https://3g.163.com/news/article/GH7D2FAD000189FH.html?clickfrom=channel2018_news_newsList#offset=2)

南京开展江宁部分区域第7轮核酸检测 (链接：https://3g.163.com/news/article/GH7A0T0B051497H3.html?clickfrom=channel2018_news_newsList#offset=3)

丈夫打赏女主播40万 患癌离世前告诉妻子:儿子靠你了 (链接：https://3g.163.com/news/article/GH794GS30512BEVO.html?clickfrom=channel2018_news_index_newslist#child=index&offset=27420)

村党委书记夫妇遇袭一死一伤  嫌疑人落网时满身血迹 (链接：https://3g.163.com/news/article/GH7S4IL3051492T3.html?clickfrom=channel2018_news_index_newslist#child=index&offset=27421)

于月仙遗体告别仪式举行 赵本山夫人等数十位名人痛悼 (链接：https://3g.163.com/news/article/GH91SV7F0514D3UH.html?clickfrom=channel2018_news_index_newslist#child=index&offset=27422)

巴方通报9名中国人遇袭身亡调查进展 中方深夜回应 (链接：https://3g.163.com

In [11]:
# 演示：写入到csv文件（采用DictWriter方式）
# 务必先浏览下参考资料，以下是官方文档链接
# ref: https://docs.python.org/3.7/library/csv.html

from csv import DictWriter

with open("news.csv", "w", encoding="utf-8", newline='') as fp:  # 记得要加newline=''；官方文档中有说明
    w = DictWriter(fp, fieldnames=['新闻标题', '链接'])
    w.writeheader()  # 写入表头
    for title, link in zip(news_titles, news_links):
        w.writerow({
            '新闻标题': title,
            '链接': link,
        })

# 执行完成后，用EmEditor、UltraEdit等软件去查看news.csv

#### 例子3：把抓取到的内容写入写入文件型数据库SQLite

In [13]:
# 演示：写入到sqlite数据库
# 务必先浏览下参考资料，以下是官方文档链接
# ref: https://docs.python.org/3.7/library/sqlite3.html
# sqlite3这个模块是根据官方的DB-API 2.0规范开发的；
# 改用其他关系型数据库时，如果也是根据DB-API规范来做的，那么使用方式几乎是完全一样的，
# 只是各家数据库SQL语法可能不大一样，需要改改SQL而已

import sqlite3

# 为了重复运行、演示，先删除已有的数据库文件
try:
    os.remove("news.db")
except FileNotFoundError:
    # 忽略文件不存在的错误
    pass

conn = sqlite3.connect("news.db")

In [14]:
c = conn.cursor()  # 获取游标（有游标才能执行sql查询、通过游标获取数据，等等）

c.execute("""
CREATE TABLE news (title text, link text)
""")  # 一开始什么表都没有，要先建立表结构

In [15]:
c.executemany("""
INSERT INTO news VALUES (?,?)
""", zip(news_titles, news_links))  # 批量把数据插入数据库表中
conn.commit()  # “提交”表示确认本次的修改操作

In [16]:
c.execute("""
SELECT * FROM news
""")  # 把我们放进去的数据查询出来验证下看看

c.fetchall()

[('治国理政新实践 | 绿色绘出幸福底色',
  'https://3g.163.com/news/article/GH95GLOJ000189FH.html?clickfrom=channel2018_news_newsList#offset=0'),
 ('美国是当之无愧的全球第一抗疫失败国',
  'https://3g.163.com/news/article/GH95I3F0000189FH.html?clickfrom=channel2018_news_newsList#offset=1'),
 ('乌合麒麟发布新作，每个细节都是梗！',
  'https://3g.163.com/news/article/GH7D2FAD000189FH.html?clickfrom=channel2018_news_newsList#offset=2'),
 ('南京开展江宁部分区域第7轮核酸检测',
  'https://3g.163.com/news/article/GH7A0T0B051497H3.html?clickfrom=channel2018_news_newsList#offset=3'),
 ('丈夫打赏女主播40万 患癌离世前告诉妻子:儿子靠你了',
  'https://3g.163.com/news/article/GH794GS30512BEVO.html?clickfrom=channel2018_news_index_newslist#child=index&offset=27420'),
 ('村党委书记夫妇遇袭一死一伤  嫌疑人落网时满身血迹',
  'https://3g.163.com/news/article/GH7S4IL3051492T3.html?clickfrom=channel2018_news_index_newslist#child=index&offset=27421'),
 ('于月仙遗体告别仪式举行 赵本山夫人等数十位名人痛悼',
  'https://3g.163.com/news/article/GH91SV7F0514D3UH.html?clickfrom=channel2018_news_index_newslist#child=index&offset=27422'),
 ('巴方通报9名中国人

In [17]:
# 完事最后别忘了清理游标，关闭数据库的连接
c.close()
conn.close()

#### 作业题：

抓取网易新闻首页中的多条新闻，仅抓取那些 每一块的左边是标题、XXX时间、XXX跟帖 右边附带一张封面图的 那些新闻；

抓取每篇文章的标题、链接、发布日期、媒体名称、跟帖数量、正文内容（仅文字内容 不包含图片视频等）、封面图片；

作业结果要求：
- 新闻存放的格式是，以新闻标题起名文件夹，每个文件夹中包含一个文本文件记录这篇文章上述的文本内容，以及封面图片的文件
- 在这些新闻标题文件夹的外面，生成一个汇总 标题、链接、发布日期、媒体名称、跟帖数量 的csv表格文件

提供抓取的首页链接（还是那个）：https://3g.163.com/touch/news

首页新闻项的参考XPath：
`//div[contains(@class, 'tab-content')]//article`